## Creating oscilloscope and rotating base objects

In [ ]:
from osctck import *
from rotmcESP import *
from utils import *
from usrt import *

TDS2024 = Osctck('USB0::1689::874::C034414::0::INSTR')
ESP300 = RotmcESP('ASRL/dev/ttyUSB0::INSTR')

## Instrument configuration

In [ ]:
TDS2024.config(channels = (1,),
                 triggerSource = 'EXT',
                 triggerLevel = -0.4,
                 triggerSlope = 'FALL',
                 triggerMode = 'NORM',
                 triggerCoup = 'AC',
                 acquisition = 128,
                 vAutoScale = False)

ESP300.config(axis = 2,
              vel = 2,
              direction = '+',
              setOrigin = True)

## Measurement cycle

In [ ]:
# Rotation parameters
initAng = 0
endAng = 360
angleStep = 10
currPostion = initAng

# Data save variables
ltMeas = []
ltAng = []
filePath = getFilePath()

# Start of measurement cycle
for i in tqdm(range(initAng, endAng, angleStep)):
    meas = TDS2024()
    ltAng.append(currPostion)
    ltMeas.append(meas[1])
    currPostion = ESP300(reference = 'REL', rotAngle = angleStep)

# Data saving
sinogram = np.array([np.array(i) for i in ltMeas])
t = meas[0]
angles = np.array(ltAng)
np.savez(filePath + '.npz', tAxis=t, Sinogram=sinogram, Angles=angles)

## Synogram

In [ ]:
axis_ticks = [t[0]*1e6, t[-1]*1e6, angles[-1], angles[0]]
plt.imshow(sinogram, cmap="RdBu_r", interpolation="nearest",extent=axis_ticks, aspect="auto")
plt.xlabel("Time [μs]")
plt.ylabel("Measurement angle [°]")
plt.colorbar()
plt.show()

## Reconstruction

In [ ]:
# Transducer impulse response reading
trData = np.load('Mediciones/trData.npz')
impResp = trData['A']

# Reading saved measurements
data = np.load(filePath + '.npz')
tAxis = data['tAxis']
Sinogram = data['Sinogram']

Rs = 42.625e-3          # distance between the transducer and the center of the axis of rotation
Na = Sinogram.shape[0]  # number of angles measured
arc = 360               # arc of circumference [°]
vs = 1480               # speed of sound in the environment [m/s]
Snoise = 0.1e-4         # standard deviation of measured noise
hfrec = 5e6             # ideal filter response frequency value [Hz]
nx = 256                # number of pixels per side of the image grid
dx =  0.08e-3           # pixel size [m]
plot = True

F = usrt(Sinogram, impResp, tAxis, Snoise, hfrec, vs, nx, dx, Rs, arc, plot)